<a href="https://colab.research.google.com/github/IgnasiOliveras/anonimitzar/blob/main/LLISTA_BERT_v_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import random
import re

# Función de preprocesamiento
def preprocesar_texto(texto):
    """Convierte el texto a minúsculas y elimina signos de puntuación."""
    texto = texto.lower()
    texto = re.sub(r'[^\w\sÁÉÍÓÚáéíóúÑñ]', '', texto)
    return texto

# Paso 1: Preprocesamiento
print("Iniciando preprocesamiento...")

# Cargar datos desde el archivo Excel
df = pd.read_excel("MOSTRA_1.xlsx")

# Aplicar preprocesamiento a la columna 'body'
df["body_preprocesado"] = df["body"].astype(str).apply(preprocesar_texto)

# Guardar el archivo preprocesado
df.to_excel("MOSTRA_1_preprocesado.xlsx", index=False)

print("Preprocesamiento completado. Archivo guardado como 'MOSTRA_1_preprocesado.xlsx'")

# Paso 2: Anonimización
print("Iniciando anonimización...")

# Cargar lista de nombres desde el archivo CSV
nombres_df = pd.read_csv("2024_pad_m_nom_sexe.csv")

# Filtrar nombres por género (convertir a minúsculas y eliminar espacios extra)
nombres_masculinos = set(nombres_df[nombres_df["SEXE"] == 2]["NOM"].str.strip().str.lower().tolist())
nombres_femeninos = set(nombres_df[nombres_df["SEXE"] == 1]["NOM"].str.strip().str.lower().tolist())

# Lista de palabras comunes que no deben anonimizarse
palabras_excluidas = {"bo","d'una","dan","justo","salud","li","leo","te","ella", "el", "los", "las", "nosotros", "vosotros",
                      "usted", "ustedes", "ellos", "ellas", "su", "sus", "un", "una", "unos", "unas", "nada", "mira", "duna",
                      "rabia", "dora", "saba", "esperanza", "domingo","sol"}

# Diccionario para mantener consistencia de nombres inventados
nombre_map = {}
cambios = []  # Lista para registrar los cambios

def detectar_genero(nombre):
    """Detecta si un nombre es masculino o femenino basado en la lista cargada."""
    nombre = nombre.lower()
    if nombre in nombres_masculinos:
        return "male"
    elif nombre in nombres_femeninos:
        return "female"
    return "neutral"

def mantener_formato(original, nuevo):
    """Ajusta el formato del nuevo nombre para que coincida con el original."""
    if original.istitle():  # Primera letra mayúscula
        return nuevo.title()
    elif original.isupper():  # Todo en mayúsculas
        return nuevo.upper()
    else:  # Todo en minúsculas
        return nuevo.lower()

def generar_nombre(original_name):
    """Genera un nombre ficticio manteniendo el género y la cantidad de palabras."""
    palabras = original_name.split()
    num_palabras = len(palabras)

    if original_name in nombre_map:
        return nombre_map[original_name]

    genero = detectar_genero(palabras[0])
    nuevo_nombre = []

    for _ in range(num_palabras):
        if genero == "male":
            nuevo_nombre.append(random.choice(list(nombres_masculinos)))
        else:
            nuevo_nombre.append(random.choice(list(nombres_femeninos)))

    nombre_final = " ".join(nuevo_nombre)
    nombre_final = mantener_formato(original_name, nombre_final)

    nombre_map[original_name] = nombre_final
    return nombre_final

def anonimizar_texto(texto):
    """Reemplaza nombres en el texto por versiones anonimizadas manteniendo consistencia."""
    # Convert the input to a string before calling split
    texto = str(texto)
    palabras = texto.split()
    texto_anonimizado = []

    for i, palabra in enumerate(palabras):
        palabra_limpia = re.sub(r'[^A-Za-zÁÉÍÓÚáéíóúÑñ]', '', palabra).lower()
        if palabra_limpia in nombres_masculinos or palabra_limpia in nombres_femeninos:
            if palabra_limpia not in palabras_excluidas:
                nombre_anonimizado = generar_nombre(palabra)
                texto_anonimizado.append(nombre_anonimizado)

                # Get 3 previous words
                palabra_anterior = " ".join(palabras[i-3:i]) if i >= 3 else " ".join(palabras[:i])
                # Get 3 posterior words
                palabra_posterior = " ".join(palabras[i+1:i+4]) if i < len(palabras) - 3 else " ".join(palabras[i+1:])

                cambios.append((palabra, nombre_anonimizado, palabra_anterior, palabra_posterior))
            else:
                texto_anonimizado.append(palabra)
        else:
            texto_anonimizado.append(palabra)

    return " ".join(texto_anonimizado)
# Cargar el archivo preprocesado
df_preprocesado = pd.read_excel("MOSTRA_1_preprocesado.xlsx")

# Aplicar anonimización a la columna 'body_preprocesado'
df_preprocesado["body_anonimizado"] = df_preprocesado["body_preprocesado"].apply(anonimizar_texto)

# Guardar el archivo anonimizado
df_preprocesado.to_excel("MOSTRA_1_anonimizado.xlsx", index=False)

# Crear DataFrame con los cambios y guardarlo en un archivo separado
cambios_df = pd.DataFrame(cambios, columns=["Original", "Anonimizado", "Palabra_Anterior", "Palabra_Posterior"])
cambios_df.to_excel("cambios_anonimizacion.xlsx", index=False)

print("Anonimización completada. Archivos guardados como 'MOSTRA_1_anonimizado.xlsx' y 'cambios_anonimizacion.xlsx'")


Iniciando preprocesamiento...
Preprocesamiento completado. Archivo guardado como 'MOSTRA_1_preprocesado.xlsx'
Iniciando anonimización...
Anonimización completada. Archivos guardados como 'MOSTRA_1_anonimizado.xlsx' y 'cambios_anonimizacion.xlsx'


In [9]:
import pandas as pd
import random
import re
import spacy

!python -m spacy download es_core_news_sm

# Cargar modelo de spaCy en español
nlp = spacy.load("es_core_news_sm")

# Función de preprocesamiento
def preprocesar_texto(texto):
    """Convierte el texto a minúsculas y elimina signos de puntuación."""
    texto = texto.lower()
    texto = re.sub(r'[^\w\sÁÉÍÓÚáéíóúÑñ]', '', texto)
    return texto

# Cargar datos desde el archivo Excel
df = pd.read_excel("MOSTRA_1.xlsx")

# Aplicar preprocesamiento
df["body_preprocesado"] = df["body"].astype(str).apply(preprocesar_texto)

# Guardar preprocesado
df.to_excel("MOSTRA_1_preprocesado.xlsx", index=False)

# Cargar lista de nombres
nombres_df = pd.read_csv("2024_pad_m_nom_sexe.csv")
nombres_masculinos = set(nombres_df[nombres_df["SEXE"] == 2]["NOM"].str.strip().str.lower().tolist())
nombres_femeninos = set(nombres_df[nombres_df["SEXE"] == 1]["NOM"].str.strip().str.lower().tolist())

# Palabras que no se deben anonimizar
palabras_excluidas = {"bo", "d'una", "dan", "justo", "salud", "li", "leo", "te", "ella", "el", "los", "las",
                      "nosotros", "vosotros", "usted", "ustedes", "ellos", "ellas", "su", "sus", "un", "una", "unos",
                      "unas", "nada", "mira", "duna", "rabia", "dora", "saba", "esperanza", "domingo", "sol"}

# Diccionario para consistencia en anonimización
nombre_map = {}
cambios = []

# Funciones auxiliares
def detectar_genero(nombre):
    nombre = nombre.lower()
    if nombre in nombres_masculinos:
        return "male"
    elif nombre in nombres_femeninos:
        return "female"
    return "neutral"

def mantener_formato(original, nuevo):
    if original.istitle():
        return nuevo.title()
    elif original.isupper():
        return nuevo.upper()
    return nuevo.lower()

def generar_nombre(original_name):
    palabras = original_name.split()
    num_palabras = len(palabras)

    if original_name in nombre_map:
        return nombre_map[original_name]

    genero = detectar_genero(palabras[0])
    nuevo_nombre = []

    for _ in range(num_palabras):
        if genero == "male":
            nuevo_nombre.append(random.choice(list(nombres_masculinos)))
        else:
            nuevo_nombre.append(random.choice(list(nombres_femeninos)))

    nombre_final = " ".join(nuevo_nombre)
    nombre_final = mantener_formato(original_name, nombre_final)

    nombre_map[original_name] = nombre_final
    return nombre_final

def anonimizar_texto(texto):
    texto = str(texto)
    palabras = texto.split()
    texto_anonimizado = []

    for i, palabra in enumerate(palabras):
        palabra_limpia = re.sub(r'[^A-Za-zÁÉÍÓÚáéíóúÑñ]', '', palabra).lower()
        if palabra_limpia in nombres_masculinos or palabra_limpia in nombres_femeninos:
            if palabra_limpia not in palabras_excluidas:
                nombre_anonimizado = generar_nombre(palabra)
                texto_anonimizado.append(nombre_anonimizado)

                palabra_anterior = " ".join(palabras[i-3:i]) if i >= 3 else " ".join(palabras[:i])
                palabra_posterior = " ".join(palabras[i+1:i+4]) if i < len(palabras) - 3 else " ".join(palabras[i+1:])

                cambios.append((palabra, nombre_anonimizado, palabra_anterior, palabra_posterior))
            else:
                texto_anonimizado.append(palabra)
        else:
            texto_anonimizado.append(palabra)

    return " ".join(texto_anonimizado)

# Cargar datos preprocesados
df_preprocesado = pd.read_excel("MOSTRA_1_preprocesado.xlsx")

# Aplicar anonimización
df_preprocesado["body_anonimizado"] = df_preprocesado["body_preprocesado"].apply(anonimizar_texto)

# Guardar anonimizado
df_preprocesado.to_excel("MOSTRA_1_anonimizado.xlsx", index=False)

# Guardar cambios en nombres
cambios_df = pd.DataFrame(cambios, columns=["Original", "Anonimizado", "Palabra_Anterior", "Palabra_Posterior"])
cambios_df.to_excel("cambios_anonimizacion.xlsx", index=False)

print("Anonimización completada. Ejecutando NER para validación...")

### 🔍 Integración con spaCy NER
def extraer_entidades(texto):
    """Extrae entidades nombradas de un texto usando spaCy."""
    doc = nlp(str(texto))
    return {ent.text for ent in doc.ents}  # Devolver conjunto de entidades únicas

# Aplicar NER en ambas versiones
df_preprocesado["entidades_original"] = df["body"].apply(extraer_entidades)
df_preprocesado["entidades_anonimizado"] = df_preprocesado["body_anonimizado"].apply(extraer_entidades)

# Comparar entidades y decidir si revertir al texto original
def restaurar_si_cambia(row):
    if row["entidades_original"] != row["entidades_anonimizado"]:
        return row["body"]  # Restaurar original si hay cambios
    return row["body_anonimizado"]  # Mantener anonimizado si no hay cambios

df_preprocesado["body_final"] = df_preprocesado.apply(restaurar_si_cambia, axis=1)

# Guardar resultado final
df_preprocesado.to_excel("MOSTRA_1_final.xlsx", index=False)

print("Proceso completo. Archivo guardado como 'MOSTRA_1_final.xlsx'")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 65.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Anonimización completada. Ejecutando NER para validación...
Proceso completo. Archivo guardado como 'MOSTRA_1_final.xlsx'


In [10]:
import pandas as pd
import random
import re
import spacy

# Cargar modelo de spaCy en español
nlp = spacy.load("es_core_news_sm")

# Cargar datos
df = pd.read_excel("MOSTRA_1_anonimizado.xlsx")
df_original = pd.read_excel("MOSTRA_1.xlsx")

# 🔍 Función para extraer nombres de persona
def extraer_nombres_persona(texto):
    """Extrae solo nombres de persona detectados por spaCy."""
    doc = nlp(str(texto))
    return {ent.text for ent in doc.ents if ent.label_ == "PER"}  # Filtrar solo entidades de persona

# Aplicar extracción de nombres de persona
df["nombres_original"] = df_original["body"].apply(extraer_nombres_persona)
df["nombres_anonimizado"] = df["body_anonimizado"].apply(extraer_nombres_persona)

# Guardar el resultado en un archivo Excel
df.to_excel("MOSTRA_1_nombres_detectados.xlsx", index=False)

# Mostrar los primeros resultados
print(df[["nombres_original", "nombres_anonimizado"]].head())

print("Proceso completado. Archivo guardado como 'MOSTRA_1_nombres_detectados.xlsx'")


  nombres_original     nombres_anonimizado
0               {}                      {}
1     {Bienvenido}  {benvingutda, tatenem}
2               {}                      {}
3               {}                      {}
4               {}                      {}
Proceso completado. Archivo guardado como 'MOSTRA_1_nombres_detectados.xlsx'


In [16]:
import pandas as pd
import random
import re
import torch
import time
from transformers import AutoModelForTokenClassification, AutoTokenizer, pipeline

# 🚀 Iniciar timer global
inicio_total = time.time()

# 🚀 Cargar modelo NER con XLM-RoBERTa
print("🔹 Cargando modelo NER...")
inicio = time.time()
modelo_ner = "MMG/xlm-roberta-large-ner-spanish"
tokenizer = AutoTokenizer.from_pretrained(modelo_ner)
modelo = AutoModelForTokenClassification.from_pretrained(modelo_ner)
nlp_ner = pipeline("ner", model=modelo, tokenizer=tokenizer, aggregation_strategy="simple")
print(f"✅ Modelo cargado en {time.time() - inicio:.2f} segundos.\n")

# 📌 Función para extraer nombres de persona con XLM-RoBERTa
def extraer_nombres_ner(texto):
    """Extrae nombres de persona usando XLM-RoBERTa."""
    if not isinstance(texto, str) or texto.strip() == "":
        return set()
    entidades = nlp_ner(texto)
    return {ent["word"] for ent in entidades if ent["entity_group"] == "PER"}

# 📌 Función de preprocesamiento
def preprocesar_texto(texto):
    return re.sub(r'[^\w\sÁÉÍÓÚáéíóúÑñ]', '', texto.lower())

# 📂 Cargar datos
print("🔹 Cargando datos...")
inicio = time.time()
df = pd.read_excel("MOSTRA_1.xlsx")
df["body_preprocesado"] = df["body"].astype(str).apply(preprocesar_texto)
print(f"✅ Datos cargados en {time.time() - inicio:.2f} segundos.\n")

# 📌 Cargar lista de nombres
print("🔹 Cargando lista de nombres...")
inicio = time.time()
nombres_df = pd.read_csv("2024_pad_m_nom_sexe.csv")
nombres_masculinos = set(nombres_df[nombres_df["SEXE"] == 2]["NOM"].str.strip().str.lower().tolist())
nombres_femeninos = set(nombres_df[nombres_df["SEXE"] == 1]["NOM"].str.strip().str.lower().tolist())
print(f"✅ Lista de nombres cargada en {time.time() - inicio:.2f} segundos.\n")

# 📌 Palabras que no se deben anonimizar
palabras_excluidas = {"bo", "d'una", "justo", "salud", "leo", "te", "ella", "el", "los", "las"}

# 📌 Diccionario para mantener consistencia en la anonimización
nombre_map = {}
cambios = []

def detectar_genero(nombre):
    nombre = nombre.lower()
    return "male" if nombre in nombres_masculinos else "female" if nombre in nombres_femeninos else "neutral"

def mantener_formato(original, nuevo):
    return nuevo.title() if original.istitle() else nuevo.upper() if original.isupper() else nuevo.lower()

def generar_nombre(original_name):
    if original_name in nombre_map:
        return nombre_map[original_name]
    genero = detectar_genero(original_name.split()[0])
    nuevo_nombre = random.choice(list(nombres_masculinos if genero == "male" else nombres_femeninos))
    nombre_map[original_name] = mantener_formato(original_name, nuevo_nombre)
    return nombre_map[original_name]

# 🔍 Aplicar anonimización y registrar cambios
print("🔹 Iniciando anonimización...")
inicio = time.time()
total_filas = len(df)
df["body_anonimizado"], df["cambio_realizado"] = zip(*df["body_preprocesado"].apply(lambda x: anonimizar_texto(x)))

# ⏳ Tiempo total estimado
tiempo_anonimizacion = time.time() - inicio
print(f"✅ Anonimización completada en {tiempo_anonimizacion:.2f} segundos.\n")

# 🔍 Aplicar NER **solo en textos donde hubo cambios**
print("🔹 Aplicando NER solo en textos modificados...")
inicio = time.time()
df["nombres_original"] = df.apply(lambda row: extraer_nombres_ner(row["body"]) if row["cambio_realizado"] else set(), axis=1)
df["nombres_anonimizado"] = df.apply(lambda row: extraer_nombres_ner(row["body_anonimizado"]) if row["cambio_realizado"] else set(), axis=1)
print(f"✅ NER aplicado en {time.time() - inicio:.2f} segundos.\n")

# 📌 Comparar nombres detectados antes y después de la anonimización
print("🔹 Comparando nombres originales y anonimizados...")
inicio = time.time()
df["body_final"] = df.apply(lambda row: row["body"] if row["nombres_original"] != row["nombres_anonimizado"] else row["body_anonimizado"], axis=1)
print(f"✅ Comparación completada en {time.time() - inicio:.2f} segundos.\n")

# 📂 Guardar resultado final
print("🔹 Guardando resultado final...")
inicio = time.time()
df.to_excel("MOSTRA_1_final.xlsx", index=False)
print(f"✅ Archivo guardado en {time.time() - inicio:.2f} segundos.\n")

# ⏳ Tiempo total de ejecución
print(f"⏳ Tiempo total de ejecución: {time.time() - inicio_total:.2f} segundos.")





🔹 Cargando modelo NER...


Device set to use cpu


✅ Modelo cargado en 6.16 segundos.

🔹 Cargando datos...
✅ Datos cargados en 4.64 segundos.

🔹 Cargando lista de nombres...
✅ Lista de nombres cargada en 0.01 segundos.

🔹 Iniciando anonimización...
✅ Anonimización completada en 0.10 segundos.

🔹 Aplicando NER solo en textos modificados...
✅ NER aplicado en 851.36 segundos.

🔹 Comparando nombres originales y anonimizados...
✅ Comparación completada en 0.09 segundos.

🔹 Guardando resultado final...
✅ Archivo guardado en 5.58 segundos.

⏳ Tiempo total de ejecución: 867.95 segundos.
